In [1]:
import os

import pandas as pd

In [2]:
def parse_order(x):
    series = pd.Series()

    series['products'] = '_'.join(x['product_id'].values.astype(str).tolist())
    series['reorders'] = '_'.join(x['reordered'].values.astype(str).tolist())
    series['aisles'] = '_'.join(x['aisle_id'].values.astype(str).tolist())
    series['departments'] = '_'.join(x['department_id'].values.astype(str).tolist())

    series['order_number'] = x['order_number'].iloc[0]
    series['order_dow'] = x['order_dow'].iloc[0]
    series['order_hour'] = x['order_hour_of_day'].iloc[0]
    series['days_since_prior_order'] = x['days_since_prior_order'].iloc[0]

    return series

In [4]:
def parse_user(x):
    parsed_orders = x.groupby('order_id', sort=False).apply(parse_order)

    series = pd.Series()

    series['order_ids'] = ' '.join(parsed_orders.index.map(str).tolist())
    series['order_numbers'] = ' '.join(parsed_orders['order_number'].map(str).tolist())
    series['order_dows'] = ' '.join(parsed_orders['order_dow'].map(str).tolist())
    series['order_hours'] = ' '.join(parsed_orders['order_hour'].map(str).tolist())
    series['days_since_prior_orders'] = ' '.join(parsed_orders['days_since_prior_order'].map(str).tolist())

    series['product_ids'] = ' '.join(parsed_orders['products'].values.astype(str).tolist())
    series['aisle_ids'] = ' '.join(parsed_orders['aisles'].values.astype(str).tolist())
    series['department_ids'] = ' '.join(parsed_orders['departments'].values.astype(str).tolist())
    series['reorders'] = ' '.join(parsed_orders['reorders'].values.astype(str).tolist())

    series['eval_set'] = x['eval_set'].values[-1]

    return series

In [5]:
orders = pd.read_csv('../data/raw/orders.csv')
prior_products = pd.read_csv('../data/raw/order_products__prior.csv')
train_products = pd.read_csv('../data/raw/order_products__train.csv')
order_products = pd.concat([prior_products, train_products], axis=0)
products = pd.read_csv('../data/raw/products.csv')

In [6]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2717275,5,prior,1,3,12,NaN
1,1909121,5,prior,2,0,16,11.0
2,2267326,5,prior,3,3,18,10.0
3,157374,5,prior,4,1,18,19.0
4,2196797,5,train,5,0,11,6.0


In [7]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,45,16349,1,1
1,53,24852,1,1
2,53,37646,2,1
3,53,47209,3,1
4,53,36606,4,0


In [9]:
df = orders.merge(order_products, how='left', on='order_id')

In [10]:
df = df.merge(products, how='left', on='product_id')

In [11]:
df['days_since_prior_order'] = df['days_since_prior_order'].fillna(0).astype(int)

In [19]:
df.isnull().sum()

order_id                     0
user_id                      0
eval_set                     0
order_number                 0
order_dow                    0
order_hour_of_day            0
days_since_prior_order       0
product_id                7500
add_to_cart_order         7500
reordered                 7500
product_name              7500
aisle_id                  7500
department_id             7500
dtype: int64

In [20]:
null_cols = ['product_id', 'aisle_id', 'department_id', 'add_to_cart_order', 'reordered']

In [21]:
df[null_cols] = df[null_cols].fillna(0).astype(int)

In [22]:
if not os.path.isdir('../data/processed'):
        os.makedirs('../data/processed')

In [23]:
user_data = df.groupby('user_id', sort=False).apply(parse_user).reset_index()

In [24]:
type(user_data)

pandas.core.frame.DataFrame

In [26]:
user_data.shape

(20621, 11)

In [25]:
user_data.head()

,user_id,order_ids,order_numbers,order_dows,order_hours,days_since_prior_orders,product_ids,aisle_ids,department_ids,reorders,eval_set
0,5,2717275 1909121 2267326 157374 2196797,1 2 3 4 5,3 0 3 1 0,12 16 18 18 11,0 11 10 19 6,15349_21413_48775_28289_8518_11777_31717_26604...,29_66_83_123_83_123_16_24_84_83_120 84_12_120_...,13_6_4_4_4_4_4_4_16_4_16 16_9_16_4_4_13_4_19_4...,0_0_0_0_0_0_0_0_0_0_0 1_0_1_1_0_0_1_0_0 1_1_0_...,train
1,42,940313 1816335 19190 77399 2848715 1560102 232...,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17,2 1 1 1 2 1 3 0 3 5 3 2 4 5 0 0 0,10 15 19 9 18 19 13 10 19 1 18 13 12 16 15 13 11,0 6 7 7 8 13 9 11 24 9 19 20 9 22 30 14 14,9185_24098_32599_2627_34594 24098_43693_79_184...,126_90_11_133_84 90_84_95_112_21_130_72_31_31 ...,11_7_11_11_16 7_16_15_3_16_14_13_7_7 15_1_1_1_...,0_0_0_0_0 1_0_0_0_0_0_0_0_0 1_0_0_0_0_0 1_1_1_...,train
2,52,2497897 1318871 1261384 2100631 580568 1680554...,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,1 1 1 1 2 1 2 1 1 2 2 1 1 1 2 3 2 2 3 1 5 1 1 ...,9 10 10 11 10 13 7 10 11 10 9 16 10 13 11 11 1...,0 7 7 7 8 6 8 6 7 8 7 6 7 7 8 8 6 14 8 5 30 10...,12341_14444_10441_196_46149_35561 12341_6184_1...,32_117_117_77_77_107 32_32_117_77_77_117 77_11...,4_19_19_7_7_19 4_4_19_7_7_19 7_19_7_4_9 4_19_1...,0_0_0_0_0_0 1_0_1_1_1_0 1_1_1_0_0 1_1_1_1_0_1_...,train
3,93,2319586 1144929 2482337 1154181 983132 2852868...,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15,3 2 5 6 5 5 6 0 6 0 6 6 4 3 1,8 9 15 9 9 9 13 11 10 12 14 8 12 19 10,0 30 30 1 6 0 1 1 20 8 27 14 5 30 5,46585 48142_3389_31225 31225 31225 35458_17341...,30 77_77_43 43 43 93_93_112_43_58_112 45_45 10...,6 7_7_3 3 3 3_3_3_3_1_3 19_19 19 19 14_3_9_14 ...,0 0_0_0 1 1 0_0_0_1_0_0 0_0 0 1 0_1_0_0 0 0 0_...,train
4,99,2736359 465262 2872285 828701 2216860 1262038 ...,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19,4 4 0 2 1 3 2 3 2 0 5 3 4 4 5 5 0 1 2,15 13 17 13 11 13 12 15 20 16 10 15 16 16 10 1...,0 14 24 23 13 23 13 29 30 12 26 12 15 21 8 7 2...,27845_23516_23726_23405_43920_32666_13802_4920...,84_17_17_17_17_17_36_123_117_117_19 24_123_26_...,16_13_13_13_13_13_16_4_19_19_13 4_4_7_15_4_19_...,0_0_0_0_0_0_0_0_0_0_0 0_1_0_0_0_0_0 1_1_1_0_0_...,train


In [27]:
user_data.to_csv('../data/processed/user_data.csv', index=False)